<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# OpenAI - Generate_Act_as_a_x_notebook
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/OpenAI/OpenAI_Generate_Act_as_a_x_notebook.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OpenAI+-+Generate_Act_as_a_x_notebook:+Error+short+description">Bug report</a>

**Tags:** #openai #ai #machinelearning #deeplearning #notebooks #automation #gsheet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-06-14 (Created: 2023-06-14)

**Description:** This notebook creates "Act as a ..." notebooks from a Google Sheets spreadsheet using OpenAI_Act_as_a_chef.ipynb as template.

**References:**
- [OpenAI Documentation](https://openai.com/docs/)
- [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts#act-as-a-chef)

## Input

### Import libraries

In [ ]:
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
import naas
from naas_drivers import gsheet
import copy
import json
import subprocess
try:
    from googlesearch import search
except:
    !pip install google
    from googlesearch import search
import re

### Setup variables

In [ ]:
# Inputs
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1UpUo-4IjhUxzjk0-nDCWmKoYu2e1JQ5SHRISGvZRmM8/edit#gid=0"
sheet_name = "Ref"

# Outputs
notebook_init = "OpenAI_Act_as_Stephen_Curry.ipynb"
fullname_init = "Stephen Curry"
wikipedia_init = "https://en.wikipedia.org/wiki/Stephen_Curry"
avatar_init = "https://media.discordapp.net/attachments/1084579666175729694/1164235876482699335/jeymassa_Stephen_Curry_from_Golden_State_Warrios_profile_pic_63841766-5a9f-4fa7-a550-180c773abfac.png?ex=65427a27&is=65300527&hm=6a7acdb23b1e458a30e2b27469bbdfc6ec04ec03be4ba666382427b33b42051b&=&width=352&height=352"
# name_init = "Act as Stephen Curry"
# model = "gpt-3.5-turbo-16k"
# temperature = 0.5

## Model

### Open template notebook

In [ ]:
# Get notebook
nb_init = load_notebook_node(notebook_init)
references_init = nb_init.cells[6].source
variables_init = nb_init.cells[11].source
print(references_init)
print(variables_init)

### Get data from Google Sheets spreadsheet

In [ ]:
df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Row fetched:", len(df_input))
df_input.head(1)

## Output

### Get wikipedia

In [ ]:
def get_wikipedia_url(fullname):
    # Init
    url = None

    # Create query
    query = f"{fullname.replace(' ', '+')}+Wikipedia"
    print("Google query: ", query)

    # Search in Google
    for i in search(query, tld="com", num=10, stop=10, pause=2):
        pattern = "https:\/\/en.wikipedia.org\/wiki\/.([^?])+"
        result = re.search(pattern, i)

        # Return value if result is not None
        if result != None:
            url = result.group(0).replace(" ", "")
            return url
    return url

wikipedia_url = get_wikipedia_url(fullname_init)
print("Wikipedia URL:", wikipedia_url)

### Create notebooks

In [ ]:
for row in df_input.itertuples():
    # Init
    nb = copy.deepcopy(nb_init)
    character = row.Character
    avatar = row.Avatar
    description = row.Description
    print("Started with:", character)
    
    # Prep outputs
    notebook_output = f"OpenAI_Act_as_{character.replace(' ', '_')}.ipynb"
    title = f"# OpenAI - Act as {character}"
    nb_description = f"**Description:** {description}"
    wikipedia_url = get_wikipedia_url(character)
    print("Wikipedia URL:", wikipedia_url)
    
    # Update notebook
    nb.cells[1].source = title
    nb.cells[5].source = nb_description
    nb.cells[6].source = references_init.replace(fullname_init, character).replace(wikipedia_init, wikipedia_url)
    nb.cells[11].source = variables_init.replace(fullname_init, character).replace(avatar_init, avatar)
    
    # Save new notebook
    write_ipynb(nb, notebook_output)
    print("💾 Notebook saved:", notebook_output)
    
    # Run notebook
    %run $notebook_output
    print("✅ Notebook ran.")
    break